In [1]:
from WaveNetClassifier import WaveNetClassifier
import create_csv
from utils import get_audio_config
import emotion_recognition
from keras.utils import to_categorical

Using TensorFlow backend.


In [2]:
train_desc_files = ['train_custom.csv', 'train_emo.csv', 'train_tess_ravdess.csv']
test_desc_files = ['test_custom.csv', 'test_emo.csv', 'test_tess_ravdess.csv']
ALL_FEATURES = ['mfcc', 'chroma', 'mel', 'contrast', 'tonnetz', 'rmse']
image_features = ['mel', 'tonnetz', 'chroma', 'image']
non_image_features = ['contrast', 'rmse', 'mfcc']
emotions = ['happy', 'sad']

In [3]:
create_csv.write_emodb_csv(emotions=emotions, train_name="train_emo.csv",
                    test_name="test_emo.csv", train_size=0.8, verbose=1)
create_csv.write_tess_ravdess_csv(emotions=emotions, train_name="train_tess_ravdess.csv",
                            test_name="test_tess_ravdess.csv", verbose=1)
create_csv.write_custom_csv(emotions=emotions, train_name="train_custom.csv", test_name="test_custom.csv",
                    verbose=1)
int2emotions = {i: e for i, e in enumerate(emotions)}
emotions2int = {v: k for k, v in int2emotions.items()}

[EMO-DB] Total files to write: 216
[EMO-DB] Training samples: 172
[EMO-DB] Testing samples: 43
[TESS&RAVDESS] There are 655 training audio files for category:happy
[TESS&RAVDESS] There are 115 testing audio files for category:happy
[TESS&RAVDESS] There are 668 training audio files for category:sad
[TESS&RAVDESS] There are 114 testing audio files for category:sad


In [4]:
audio_config = get_audio_config(['raw'])
data_raw = emotion_recognition.load_data(train_desc_files, test_desc_files, audio_config, classification=True,
                                emotions=emotions, balance=True, shuffle=False)

In [5]:
X_train_raw = data_raw["X_train"]
X_test_raw = data_raw["X_test"]
y_train = to_categorical([emotions2int[str(e)] for e in data_raw['y_train'].squeeze() ])
y_test =  to_categorical([emotions2int[str(e)] for e in data_raw['y_test'].squeeze() ])


In [6]:
wnc = WaveNetClassifier((45158,), (2,), 
                        kernel_size = 2, dilation_depth = 9, n_filters = 40, 
                        task = 'classification')


W0828 11:27:09.869683 140735990793088 deprecation_wrapper.py:119] From /usr/local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0828 11:27:09.891111 140735990793088 deprecation_wrapper.py:119] From /usr/local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0828 11:27:10.437663 140735990793088 deprecation_wrapper.py:119] From /usr/local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0828 11:27:10.455764 140735990793088 deprecation_wrapper.py:119] From /usr/local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:3980: The name tf.nn.avg_pool is deprecated. Please use tf.nn.avg_pool2d instead.



__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
original_input (InputLayer)     (None, 45158)        0                                            
__________________________________________________________________________________________________
reshaped_input (Reshape)        (None, 45158, 1)     0           original_input[0][0]             
__________________________________________________________________________________________________
dilated_conv_1 (Conv1D)         (None, 45158, 40)    120         reshaped_input[0][0]             
__________________________________________________________________________________________________
dilated_conv_2_tanh (Conv1D)    (None, 45158, 40)    3240        dilated_conv_1[0][0]             
__________________________________________________________________________________________________
dilated_co

In [ ]:
wnc.fit(X_train_raw, y_train, validation_data = (X_test_raw, y_test), epochs = 500, 
        batch_size = 1024, optimizer='adam', save=True, save_dir='./logs/')


W0828 11:27:10.621568 140735990793088 deprecation_wrapper.py:119] From /usr/local/lib/python3.7/site-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0828 11:27:10.639727 140735990793088 deprecation_wrapper.py:119] From /usr/local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:3295: The name tf.log is deprecated. Please use tf.math.log instead.

W0828 11:27:10.721019 140735990793088 deprecation.py:323] From /usr/local/lib/python3.7/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0828 11:27:11.498309 140735990793088 deprecation_wrapper.py:119] From /usr/local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:986: The name tf.assign_add is deprecated

Train on 3804 samples, validate on 936 samples
Epoch 1/500


In [ ]:
y_pred = wnc.predict(X_test_raw)